In [2]:
import pandas as pd
import numpy as np
import random

h = 3
s = 5
c = 0.98
itr = int(input("Enter number of iterations: "))
#itr = 50
d=2
l_lim = -5.12
u_lim = 5.12

arr = np.random.uniform(l_lim , u_lim,size=(2,3))

global_best = float('inf')   
all_f_vals = []
global_best_progress = []

for z in range(itr):
    #print(arr)
    arr_x1 = arr + c
    arr_x2 = arr - c 
    arr_x1 = np.clip(arr_x1, l_lim, u_lim)
    arr_x2 = np.clip(arr_x2, l_lim, u_lim)

    #print(arr_x1)
    #print(arr_x2)

    #pm = [[[arr_x1[0][0], arr_x2[0][0]], [arr_x1[0][1], arr_x2[0][1]], [arr_x1[0][2], arr_x2[0][2]]],
    # [[arr_x1[1][0], arr_x2[1][0]], [arr_x1[1][1], arr_x2[1][1]], [arr_x1[1][2], arr_x2[1][2]]]]

    #result = np.stack((arr_x1, arr_x2), axis=2)
    #print(arr_x1[1][0])
    #print(pm)

    h1x1 = np.random.uniform(arr_x1[0][0], arr_x2[0][0],size=(s,1))
    h1x2 = np.random.uniform(arr_x1[0][1], arr_x2[0][1],size=(s,1))
    h2x1 = np.random.uniform(arr_x1[0][2], arr_x2[0][2],size=(s,1))
    h2x2 = np.random.uniform(arr_x1[1][0], arr_x2[1][0],size=(s,1))
    h3x1 = np.random.uniform(arr_x1[1][1], arr_x2[1][1],size=(s,1))
    h3x2 = np.random.uniform(arr_x1[1][2], arr_x2[1][2],size=(s,1))

    #print(h1x1)
    #print(h1x2)
    #print(h2x1)
    #print(h2x2)
    #print(h3x1)
    #print(h3x2)

    h1 = np.stack((h1x1, h1x2), axis=2)
    h2 = np.stack((h2x1 , h2x2), axis=2)
    h3 = np.stack((h3x1 , h3x2), axis=2)

    #print(h1)
    #print(h2)
    #print(h3)

    f_vals1 = np.sum(h1**2, axis=2)
    f_vals2 = np.sum(h2**2, axis=2)
    f_vals3 = np.sum(h3**2, axis=2)

    f_vals = np.stack((f_vals1 , f_vals2 , f_vals3 ) , axis = 2)
    all_f_vals.append(f_vals.copy())

    #print(f_vals)
    # ISH

    if z >= 2:
        f_t   = all_f_vals[z]
        f_t1  = all_f_vals[z-1]
        f_t2  = all_f_vals[z-2]

        numerator   = f_t - f_t1
        denominator = f_t - f_t2
        denominator[denominator == 0] = 1e-8
        ish = abs(numerator / denominator)
    else:
        ish = np.random.random(size=(s,h))
    #print(ish)

    fitness = np.stack((f_vals1 , f_vals2 , f_vals3), axis=2)
    #print(fitness)
    fitness = np.squeeze(fitness, axis=1) 
    #print(fitness1)

    #Ps = np.random.randint(1, 15,size=(s,h))
    #print(Ps)

    fitness_1 = 1 / (fitness + 0.0000000001)
    Ps = fitness_1 / np.sum(fitness_1)
    #print(np.sum(Ps))
    #print(Ps.shape)
    Ps = Ps.flatten()
    #print(Ps)
    #print(Ps.shape)
    #print("Fitness:\n", fitness)
    #print("Probabilities:\n", Ps)
    probsum = np.sum(Ps)
    #print(probsum)
    indices = np.arange(0, 15)
    #print(indices)
    selected = np.random.choice(indices, size=15, p=Ps)
    #print(selected)
    while any(selected[b] == b for b in range(h*s)):
        selected = np.random.choice(indices, size=15, p=Ps)

    #print(f_vals)

    fun_vals_f = f_vals.T
    fun_vals_f = fun_vals_f.flatten()
    #print(f_vals_f)
    #print(f_vals)
    #print(selected)
    #print(f_vals_f)
    fx_fen = np.zeros(len(selected))
    #print(fx_fen)
    fx_fen = np.array([fun_vals_f[i] for i in selected])
    #print(ish)
    ish_d=1/ish
    ish_d = ish_d.T
    ish_d = ish_d.flatten()
    #print(ish_d)
    #print(fx_fen)
    denom_ld = np.array([fun_vals_f[a] - fx_fen[a] for a in range(h*s)])
    #print(denom_ld)

    denom_ld_d = 1/(denom_ld + 0.00000001)

    #print(denom_ld_d)

    LD = np.array([ish_d[a] * denom_ld_d[a] for a in range(h*s)])
    #print(LD)

    def min_max_normalize(LD):
        min_val = np.min(LD)
        max_val = np.max(LD)
        normalized_array = (LD - min_val) / (max_val - min_val)
        return normalized_array

    #print(h1)
    #print(h2)
    #print(h3)

    #s_pos = np.stack((h1 , h2 , h3) , axis = 0)
    #print(s_pos)
    #s_pos = s_pos.flatten()
    #print(s_pos)
    NLD = min_max_normalize(LD)
    #print(NLD)
    # Fix ONLY exact 0s and 1s
    NLD[NLD == 0] += 1e-8
    NLD[NLD == 1] -= 1e-8
    #print(NLD)

    #temp = np.array([[h1x1, h1x2]])

    # Create DataFrame
    #df1 = pd.DataFrame.from_records(temp, columns=['X1', 'X2'])

    #print(df)

    #print(h3x1)
    #print(h3x2)

    #print(f_vals)
    #print(fun_vals_f)
    h1_l = [[h1x1[0],h1x2[0] ], [h1x1[1],h1x2[1]], 
           [h1x1[2],h1x2[2]], [h1x1[3],h1x2[3]]
           , [h1x1[4],h1x2[4]]]
    h2_l = [[h2x1[0],h2x2[0] ], [h2x1[1],h2x2[1]], 
           [h2x1[2],h2x2[2]], [h2x1[3],h2x2[3]],[h2x1[4],h2x2[4]]]
    h3_l = [[h3x1[0],h3x2[0] ], [h3x1[1],h3x2[1]], 
           [h3x1[2],h3x2[2]], [h3x1[3],h3x2[3]],[h3x1[4],h3x2[4]]]
    h1_h2_h3_l = [[h1x1[0],h1x2[0] ], [h1x1[1],h1x2[1]], 
           [h1x1[2],h1x2[2]], [h1x1[3],h1x2[3]]
           , [h1x1[4],h1x2[4]],[h2x1[0],h2x2[0] ], [h2x1[1],h2x2[1]], 
           [h2x1[2],h2x2[2]], [h2x1[3],h2x2[3]],[h2x1[4],h2x2[4]],[h3x1[0],h3x2[0] ], [h3x1[1],h3x2[1]], 
           [h3x1[2],h3x2[2]], [h3x1[3],h3x2[3]],[h3x1[4],h3x2[4]]]

    #print(h1_l)
    df_h1 = pd.DataFrame(h1_l, columns =['X1', 'X2'])
    df_h2 = pd.DataFrame(h2_l, columns =['X1', 'X2'])
    df_h3 = pd.DataFrame(h3_l, columns =['X1', 'X2'])
    df_h1_h2_h3 = pd.DataFrame(h1_h2_h3_l, columns =['X1', 'X2'])
    #print(df_h3.head(1))
    #print(df_h3.head(1)**2)
    #print(df_h1_h2_h3)
    #print(NLD)
    #print(NLD[14])
    #print(selected)     
    Sh_up = pd.DataFrame(columns =['X1', 'X2'])
    #print(Sh_up)
    #NLD
    #fun_vals_f
    #df_h1_h2_h3
    #NLD*(Sh - Sh_fecund)

    #print(f_vals)

    # Updated positions   Sh_up
    for m in range(h*s):
        #print(fun_vals_f[m])
        result = NLD[m]*(df_h1_h2_h3.iloc[m] - df_h1_h2_h3.iloc[selected[m]])
        Sh_up.loc[len(Sh_up)] = result.values

    #print(Sh_up)
    Sh_up_fn = np.sum(Sh_up**2, axis=1)
    Sh_up_fn = Sh_up_fn.to_frame(name='Function_Values')
    vals = Sh_up_fn.values.flatten().tolist()
    #print(Sh_up_fn)
    Sh_upnfn = pd.concat([Sh_up, Sh_up_fn], axis=1)
    #df_h1_h2_h3['Sh_up'] = Sh_up_fn.values.flatten()


    #print(vals1)

    Sh_upnfn['H'] = np.random.randint(0, 3, size=len(Sh_upnfn))

    #print(Sh_upnfn)

    Sh_upnfn['Function_Values'] = Sh_upnfn['Function_Values'].apply(lambda x: x[0])

    min_vals = Sh_upnfn.groupby('H')['Function_Values'].min()

    best = min_vals.min()
    if best < global_best:
        global_best = best
    global_best_progress.append(global_best)
    min_rows = Sh_upnfn.loc[Sh_upnfn.groupby('H')['Function_Values'].idxmin()]

    min_rows['X1'] = min_rows['X1'].apply(lambda x: x[0])
    min_rows['X2'] = min_rows['X2'].apply(lambda x: x[0])

    arr = min_rows[['X1', 'X2']].values.T

    if arr.shape[1] < h:
        missing = h - arr.shape[1]
        padding = np.random.uniform(l_lim, u_lim, size=(2, missing))
        arr = np.concatenate((arr, padding), axis=1)

    #print("Function Value:\n", min_vals) #prints best of 3 homes 
    #print("\nFunction Value:\n", min_rows)  #prints whole block of selected minimum value from all homes i.e. x1 x2 fun_val And H
    print(f"Iteration {z+1} : {best}") # prints best from 3 bests of 3 homes
print(f"Global Best : {global_best}") # prints global best from 15 snails

Iteration 1 : 1.5431277873616903e-17
Iteration 2 : 9.363402946573119e-18
Iteration 3 : 9.709578070051718e-17
Iteration 4 : 2.022819230055423e-18
Iteration 5 : 1.5543991774645559e-16
Iteration 6 : 2.2154576770406293e-16
Iteration 7 : 1.0095404220127518e-16
Iteration 8 : 1.794286292649486e-16
Iteration 9 : 1.194247564464111e-16
Iteration 10 : 5.850792423493325e-18
Iteration 11 : 4.613519942932849e-16
Iteration 12 : 1.7216643610026564e-17
Iteration 13 : 1.185121592300562e-16
Iteration 14 : 5.978164206129222e-16
Iteration 15 : 4.703896914936154e-18
Iteration 16 : 2.9055642620959377e-17
Iteration 17 : 6.279101332005871e-17
Iteration 18 : 1.7517269733820687e-16
Iteration 19 : 8.244896461099551e-17
Iteration 20 : 1.1571521619789575e-18
Iteration 21 : 2.0735313776066157e-16
Iteration 22 : 5.0793861513844566e-17
Iteration 23 : 1.978507831996305e-16
Iteration 24 : 4.450903461540933e-17
Iteration 25 : 2.344156504639278e-16
Iteration 26 : 6.985905766310491e-20
Iteration 27 : 1.316279100955626e-17
